In [1]:
%load_ext autoreload
%autoreload 2

# **Machine Learning with Linear Regression (From Scratch)**

In [2]:
import numpy as np
import plotly.graph_objects as go
from diveai.plotting import PlotBuilder

# Linear Regression with Regularization: Mathematical Foundations

This notebook explains the mathematical concepts behind our enhanced linear regression implementation supporting multiple features and regularization (Ridge, Lasso, Elastic Net).

## 1. Problem Formulation

**Given**:
- Input features matrix: $X \in \mathbb{R}^{m \times n}$  
_(m samples, n features)_
- Target vector: $y \in \mathbb{R}^m$  
- Parameters: weights $w \in \mathbb{R}^n$, bias $b \in \mathbb{R}$

**Goal**: Find parameters that minimize:
$$\min_{w,b} J(w,b) = \text{MSE} + \lambda R(w)$$

## 2. Hypothesis Function

For multiple features:
$$\hat{y}^{(i)} = w^T x^{(i)} + b = \sum_{j=1}^n w_j x_j^{(i)} + b$$

Matrix form:
$$\hat{y} = Xw + b$$

## 3. Cost Function with Regularization

**Base Cost (MSE)**:
$$J_{\text{base}} = \frac{1}{m}\sum_{i=1}^m (y^{(i)} - \hat{y}^{(i)})^2$$

**Regularization Terms**:

| Type          | Formula                          | Code Reference                     |
|---------------|----------------------------------|------------------------------------|
| **L2 (Ridge)** | $\frac{\lambda}{2m}\sum_{j=1}^n w_j^2$ | `lambda_/(2*m) * np.sum(weights**2)` |
| **L1 (Lasso)** | $\frac{\lambda}{m}\sum_{j=1}^n |w_j|$     | `lambda_/m * np.sum(np.abs(weights))` |
| **Elastic Net** | $\frac{\lambda}{m}\left( \rho\sum|w_j| + \frac{(1-\rho)}{2}\sum w_j^2 \right)$ | Combination of L1 and L2 terms |

**Total Cost**:
$$J = J_{\text{base}} + \text{Regularization Term}$$

## 4. Gradient Descent Derivation

### Base Gradients
For parameter update:
$$\theta_{\text{new}} = \theta_{\text{old}} - \alpha \frac{\partial J}{\partial \theta}$$

**Weight Gradient**:
$$
\begin{align*}
\frac{\partial J}{\partial w_j} &= \frac{2}{m}\sum_{i=1}^m (\hat{y}^{(i)} - y^{(i)})x_j^{(i)} \\
&= \frac{2}{m}X_j^T(\hat{y} - y)
\end{align*}
$$


**Bias Gradient**:
$$
\frac{\partial J}{\partial b} = \frac{2}{m}\sum_{i=1}^m (\hat{y}^{(i)} - y^{(i)})
$$


### Regularization Gradients

| Type          | Weight Gradient Additive Term   | Code Implementation              |
|---------------|----------------------------------|-----------------------------------|
| **L2 (Ridge)** | $\frac{\lambda}{m}w$            | `(lambda_/m) * self.weights`      |
| **L1 (Lasso)** | $\frac{\lambda}{m}\text{sign}(w)$ | `(lambda_/m) * np.sign(weights)` |
| **Elastic Net** | $\frac{\lambda}{m}(\rho\text{sign}(w) + (1-\rho)w)$ | Combination of L1/L2 terms |

## 5. Update Rules

**Matrix Form Updates**:


In [3]:
import numpy as np

class LinearRegression:
    def __init__(self, learning_rate=0.01, iterations=1000, dive=False,
                 regularization=None, lambda_=0.1, l1_ratio=0.5):
        """
        Enhanced Linear Regression model with regularization support
        :param learning_rate: Step size for gradient descent
        :param iterations: Number of gradient descent iterations
        :param dive: Enable detailed logging of training process
        :param regularization: Type of regularization ('l1', 'l2', 'elastic_net')
        :param lambda_: Regularization strength
        :param l1_ratio: Mixing parameter for Elastic Net (0-1)
        """
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.dive = dive
        self.regularization = regularization
        self.lambda_ = lambda_
        self.l1_ratio = l1_ratio
        self.weights = None
        self.bias = None
        
    def fit(self, X, y):
        """
        Train model with gradient descent and regularization
        """
        m, n = X.shape  # m = samples, n = features
        self.weights = np.zeros((n, 1))  # Proper shape for multiple features
        self.bias = 0
        
        y = y.reshape(-1, 1)  # Ensure proper shape
        
        # Initialize logging
        metrics = {'weights': [], 'bias': [], 'cost': []}
        
        for _ in range(self.iterations):
            # Compute predictions and errors
            y_pred = X @ self.weights + self.bias  # Matrix multiplication
            error = y_pred - y
            
            # Compute gradients (vectorized)
            dw = (2/m) * X.T @ error  # Correct gradient calculation
            db = (2/m) * np.sum(error)
            
            # Add regularization gradients
            if self.regularization == 'l2':
                dw += (self.lambda_/m) * self.weights
            elif self.regularization == 'l1':
                dw += (self.lambda_/m) * np.sign(self.weights)
            elif self.regularization == 'elastic_net':
                l1_grad = self.lambda_ * self.l1_ratio * np.sign(self.weights)
                l2_grad = self.lambda_ * (1 - self.l1_ratio) * self.weights
                dw += (l1_grad + l2_grad)/m
            
            # Update parameters
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db
            
            # Compute cost with regularization
            mse = np.mean(error**2)
            reg_cost = self._regularization_cost(m, n)
            total_cost = mse + reg_cost
            
            # Log metrics
            metrics['weights'].append(self.weights.copy())
            metrics['bias'].append(self.bias)
            metrics['cost'].append(total_cost)
        
        return metrics
    
    def _regularization_cost(self, m, n):
        """Calculate regularization term for cost function"""
        if not self.regularization:
            return 0
            
        l1_term = np.sum(np.abs(self.weights)) 
        l2_term = np.sum(self.weights**2)
        
        if self.regularization == 'l1':
            return (self.lambda_/m) * l1_term
        elif self.regularization == 'l2':
            return (self.lambda_/(2*m)) * l2_term
        elif self.regularization == 'elastic_net':
            return (self.lambda_/m) * (self.l1_ratio * l1_term + 
                                      (1 - self.l1_ratio)/2 * l2_term)
        return 0
    
    def predict(self, X):
        """Generate predictions using learned weights"""
        return X @ self.weights + self.bias


## **1. Introduction to Linear Regression**

Linear regression is a fundamental algorithm in machine learning used for predicting a continuous target variable based on one or more input features.

### **Mathematical Formulation**
The linear regression model is represented as:
$$
\hat{y} = w_0 + w_1x_1 + w_2x_2 + \dots + w_nx_n
$$
where:
- \(w_0\) is the bias (intercept),
- \(w_i\) are the weights (coefficients),
- \(x_i\) are the input features.

### **Loss Function: Mean Squared Error (MSE)**
$$
MSE = \frac{1}{N} \sum_{i=1}^N (y_i - \hat{y}_i)^2
$$

**Explanation**: MSE measures the average squared difference between predicted and actual values. It's widely used because it penalizes larger errors more heavily and is differentiable, making it suitable for optimization.

**Best Practices**:
- Use MSE when you want to penalize larger errors more heavily.
- Be cautious with outliers as they can significantly impact MSE.
- Consider Mean Absolute Error (MAE) if your data has many outliers.

## **2. Linear Regression with One Independent Variable**

### **Generating Synthetic Data**

In [4]:
np.random.seed(0)
X = 2 * np.random.rand(100, 1) # One feature
y = 4 + 3 * X + np.random.randn(100, 1) # y = 4 + 3x + noise

**Explanation**: Synthetic data allows us to understand the behavior of our algorithm in a controlled setting.

**Best Practices**:
- Use synthetic data to validate your algorithm before applying it to real-world data.
- Ensure your synthetic data mimics the properties of real data you expect to work with.

### **Visualize data**

In [5]:
pb = PlotBuilder(title="Synthetic Data for Linear Regression", x_label="Feature", y_label="Target")
pb.add_plot(X.flatten(), y.flatten(), plot_type='scatter')
pb.show()

FigureWidget({
    'data': [{'marker': {'color': 'blue', 'opacity': 1, 'size': 5},
              'mode': 'markers',
              'type': 'scatter',
              'uid': '5e7002a6-9196-4cf0-82b3-88a73d0d8d4f',
              'x': array([1.09762701, 1.43037873, 1.20552675, 1.08976637, 0.8473096 , 1.29178823,
                          0.87517442, 1.783546  , 1.92732552, 0.76688304, 1.58345008, 1.05778984,
                          1.13608912, 1.85119328, 0.14207212, 0.1742586 , 0.04043679, 1.66523969,
                          1.5563135 , 1.7400243 , 1.95723668, 1.59831713, 0.92295872, 1.56105835,
                          0.23654885, 1.27984204, 0.28670657, 1.88933783, 1.04369664, 0.82932388,
                          0.52911122, 1.54846738, 0.91230066, 1.1368679 , 0.0375796 , 1.23527099,
                          1.22419145, 1.23386799, 1.88749616, 1.3636406 , 0.7190158 , 0.87406391,
                          1.39526239, 0.12045094, 1.33353343, 1.34127574, 0.42076512, 0.2578526 ,
      

### **Hyperparameters**

In [6]:
lr = 0.01 # Learning rate
n_iters = 1000 # Number of iterations

### **Initialize weights and bias**

In [7]:
w = 0 # Weight for one feature
b = 0 # Bias

### **Gradient Descent**

In [8]:
n_samples = len(X)
mse_history = []
for _ in range(n_iters):
    y_predicted = w * X + b
    mse = np.mean((y - y_predicted)**2)
    mse_history.append(mse)

    dw = -(2 / n_samples) * np.sum(X * (y - y_predicted))
    db = -(2 / n_samples) * np.sum(y - y_predicted)

    # Update weights and bias
    w -= lr * dw
    b -= lr * db

print(f"Weight: {w}, Bias: {b}")

Weight: 2.99983630495613, Bias: 4.186799174312735


**Explanation**: Gradient Descent is an optimization algorithm used to find the weights that minimize the loss function.

**Best Practices**:
- Choose an appropriate learning rate: too high may cause divergence, too low may result in slow convergence.
- Monitor the loss during training to ensure the algorithm is converging.
- Consider using adaptive learning rate methods for more complex problems.

### **Visualize MSE over iterations**

In [9]:
pb = PlotBuilder(title="MSE vs. Iterations", x_label="Iterations", y_label="Mean Squared Error")
pb.add_plot(np.arange(n_iters), mse_history, plot_type='line', label='MSE')
pb.show()

FigureWidget({
    'data': [{'line': {'color': 'blue'},
              'marker': {'color': 'blue', 'opacity': 1, 'size': 5},
              'mode': 'lines',
              'name': 'MSE',
              'type': 'scatter',
              'uid': '925ec216-b8e1-497b-a29a-944927d0a606',
              'x': array([  0,   1,   2, ..., 997, 998, 999], shape=(1000,)),
              'y': [53.330318695242276, 49.11208555114041, 45.235080729138424,
                    ..., 0.9928051993977445, 0.9928028433141813,
                    0.9928005023748538]}],
    'layout': {'template': '...',
               'title': {'text': 'MSE vs. Iterations'},
               'xaxis': {'title': {'text': 'Iterations'}},
               'yaxis': {'title': {'text': 'Mean Squared Error'}}}
})

### **Visualizing the Best-Fit Line**

In [10]:
y_pred = w * X + b

pb = PlotBuilder(title="Linear Regression Fit", x_label="Feature", y_label="Target")
pb.add_plot(X.flatten(), y.flatten(), plot_type='scatter', label='Data')
pb.add_plot(X.flatten(), y_pred.flatten(), plot_type='line', label='Best Fit Line', color='red')
pb.show()

FigureWidget({
    'data': [{'marker': {'color': 'blue', 'opacity': 1, 'size': 5},
              'mode': 'markers',
              'name': 'Data',
              'type': 'scatter',
              'uid': 'cff918b5-c4fc-43c7-8f4c-e6e9ba233c77',
              'x': array([1.09762701, 1.43037873, 1.20552675, 1.08976637, 0.8473096 , 1.29178823,
                          0.87517442, 1.783546  , 1.92732552, 0.76688304, 1.58345008, 1.05778984,
                          1.13608912, 1.85119328, 0.14207212, 0.1742586 , 0.04043679, 1.66523969,
                          1.5563135 , 1.7400243 , 1.95723668, 1.59831713, 0.92295872, 1.56105835,
                          0.23654885, 1.27984204, 0.28670657, 1.88933783, 1.04369664, 0.82932388,
                          0.52911122, 1.54846738, 0.91230066, 1.1368679 , 0.0375796 , 1.23527099,
                          1.22419145, 1.23386799, 1.88749616, 1.3636406 , 0.7190158 , 0.87406391,
                          1.39526239, 0.12045094, 1.33353343, 1.34127574, 

**Explanation**: The best-fit line represents the model's predictions across the range of input values.

**Best Practices**:
- Always visualize your data and model predictions when working with low-dimensional data.
- Look for patterns in the residuals (differences between predictions and actual values) to identify potential issues with the model.

## **3. Extending to Multiple Features**

### **Generating Data with Multiple Features**

In [11]:
np.random.seed(42)
X_multi = np.random.rand(100, 2) # Two features
y_multi = 3 + 5 * X_multi[:, 0] + 2 * X_multi[:, 1] + np.random.randn(100)

In [12]:
pb = PlotBuilder(title="Synthetic Data with Two Features", x_label='Feature 1', y_label="Feature 2", z_label='Target')
pb.add_plot(X_multi[:, 0], X_multi[:, 1], y_multi, plot_type='scatter', label='Actual Data')
pb.show()

FigureWidget({
    'data': [{'marker': {'color': 'blue', 'opacity': 1, 'size': 2.5},
              'mode': 'markers',
              'name': 'Actual Data',
              'type': 'scatter3d',
              'uid': '4988e6e9-b288-426e-a54d-a266f8a7988d',
              'x': array([0.37454012, 0.73199394, 0.15601864, 0.05808361, 0.60111501, 0.02058449,
                          0.83244264, 0.18182497, 0.30424224, 0.43194502, 0.61185289, 0.29214465,
                          0.45606998, 0.19967378, 0.59241457, 0.60754485, 0.06505159, 0.96563203,
                          0.30461377, 0.68423303, 0.12203823, 0.03438852, 0.25877998, 0.31171108,
                          0.54671028, 0.96958463, 0.93949894, 0.59789998, 0.0884925 , 0.04522729,
                          0.38867729, 0.82873751, 0.28093451, 0.14092422, 0.07455064, 0.77224477,
                          0.00552212, 0.70685734, 0.77127035, 0.35846573, 0.86310343, 0.33089802,
                          0.31098232, 0.72960618, 0.88721274, 0

**Explanation**: Multiple features allow the model to capture more complex relationships in the data.

**Best Practices**:
- Ensure features are on similar scales or consider feature scaling.
- Be cautious of multicollinearity (high correlation between features) as it can make the model unstable.

##### **Adding Bias to the Feature Set in Linear Regression**  

**Why Add a Bias Term?** 

In linear regression, the model is:  

$$
\hat{y} = w_0 + w_1x_1 + w_2x_2 + \dots + w_nx_n
$$  

- $w_0$ (bias) allows the model to fit data that does not pass through the origin $(0,0)$.  
- Without $w_0$, the model assumes $y = 0$ when all features are zero, which may not be correct.  

**Incorporating Bias into the Feature Matrix**  

To simplify computations, we rewrite:  

$$
\hat{y} = w_0 \cdot 1 + w_1x_1 + \dots + w_nx_n
$$  

By adding a column of ones to $X$, we define:  

$$
X_{\text{bias}} =
\begin{bmatrix}
1 & x_{11} & x_{12} & \dots & x_{1n} \\
1 & x_{21} & x_{22} & \dots & x_{2n} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
1 & x_{m1} & x_{m2} & \dots & x_{mn}
\end{bmatrix}
$$  

Thus, we can represent the model as:  

$$
\hat{y} = X_{\text{bias}} \cdot w
$$  

**Implementation in Python**  


In [13]:
X_multi_bias = np.c_[np.ones((X_multi.shape[0], 1)), X_multi] # Add bias term

### **Implementing Linear Regression for Multiple Features**

In [14]:
weights = np.zeros(X_multi_bias.shape[1]) # [bias, w1, w2]

In [15]:
n_samples, n_features = X_multi_bias.shape
mse_history_multi = []
for _ in range(n_iters):
    y_predicted_multi = np.dot(X_multi_bias, weights)
    mse = np.mean((y_multi - y_predicted_multi)**2)
    mse_history_multi.append(mse)

    gradients = -(2 / n_samples) * np.dot(X_multi_bias.T, (y_multi - y_predicted_multi))

    # Update weights
    weights -= lr * gradients
    
print(f"Weights: {weights}")

Weights: [3.28043337 4.63890195 2.06241776]


In [16]:
# Create MSE vs. Iterations plot
pb = PlotBuilder(title="MSE vs. Iterations", x_label="Iterations", y_label="Mean Squared Error")
pb.add_plot(np.arange(n_iters), mse_history_multi, plot_type='line', label='MSE')
pb.show()

FigureWidget({
    'data': [{'line': {'color': 'blue'},
              'marker': {'color': 'blue', 'opacity': 1, 'size': 5},
              'mode': 'lines',
              'name': 'MSE',
              'type': 'scatter',
              'uid': 'e4fb2f93-2988-4e38-95de-9c8fa4c02d42',
              'x': array([  0,   1,   2, ..., 997, 998, 999], shape=(1000,)),
              'y': [46.06350456787621, 43.45557629865883, 41.00123254885531, ...,
                    1.0334420994178204, 1.0333063595439822, 1.0331709951395007]}],
    'layout': {'template': '...',
               'title': {'text': 'MSE vs. Iterations'},
               'xaxis': {'title': {'text': 'Iterations'}},
               'yaxis': {'title': {'text': 'Mean Squared Error'}}}
})

In [17]:
# Generate predictions for a meshgrid to visualize the plane
x1_range = np.linspace(X_multi[:, 0].min(), X_multi[:, 0].max(), 20)
x2_range = np.linspace(X_multi[:, 1].min(), X_multi[:, 1].max(), 20)
X1, X2 = np.meshgrid(x1_range, x2_range)
X1_flat, X2_flat = X1.ravel(), X2.ravel()

# Add bias term for prediction
X_grid_bias = np.c_[np.ones_like(X1_flat), X1_flat, X2_flat]
y_pred_grid = np.dot(X_grid_bias, weights).reshape(X1.shape)

In [18]:
pb = PlotBuilder(title="Multivariate Linear Regression Trend", x_label='Feature 1', y_label="Feature 2", z_label='Target')
pb.add_plot(X_multi[:, 0], X_multi[:, 1], y_multi, plot_type='scatter', label='Actual Data')
pb.add_plot(X1, X2, y_pred_grid, plot_type='surface', color='red', opacity=0.5, colorscale="Reds", label='LR Plane')
pb.show()

FigureWidget({
    'data': [{'marker': {'color': 'blue', 'opacity': 1, 'size': 2.5},
              'mode': 'markers',
              'name': 'Actual Data',
              'type': 'scatter3d',
              'uid': '6f0e499a-5f2d-4b79-8ad2-309dd2385224',
              'x': array([0.37454012, 0.73199394, 0.15601864, 0.05808361, 0.60111501, 0.02058449,
                          0.83244264, 0.18182497, 0.30424224, 0.43194502, 0.61185289, 0.29214465,
                          0.45606998, 0.19967378, 0.59241457, 0.60754485, 0.06505159, 0.96563203,
                          0.30461377, 0.68423303, 0.12203823, 0.03438852, 0.25877998, 0.31171108,
                          0.54671028, 0.96958463, 0.93949894, 0.59789998, 0.0884925 , 0.04522729,
                          0.38867729, 0.82873751, 0.28093451, 0.14092422, 0.07455064, 0.77224477,
                          0.00552212, 0.70685734, 0.77127035, 0.35846573, 0.86310343, 0.33089802,
                          0.31098232, 0.72960618, 0.88721274, 0

**Explanation**: The principle remains the same as with one feature, but now we're optimizing multiple weights simultaneously.

**Best Practices**:
- Use vectorized operations for efficiency when working with multiple features.
- Consider feature selection techniques if you have a large number of features.

## **4. Regularization Techniques**

Regularization helps prevent overfitting by adding a penalty term to the loss function.

##### **a. Ridge Regression (L2 Regularization)**
[Code for Ridge Regression]

**Explanation**: Ridge regression adds the sum of squared weights to the loss function, encouraging smaller weights.

**Best Practices**:
- Use Ridge when you suspect many features are relevant.
- It's particularly useful when you have multicollinearity in your features.
- Experiment with different alpha values to find the optimal balance between bias and variance.

##### **b. Lasso Regression (L1 Regularization)**
[Code for Lasso Regression]

**Explanation**: Lasso adds the sum of absolute weights to the loss function, which can lead to sparse models (some weights become exactly zero).

**Best Practices**:
- Use Lasso when you believe only a few features are relevant.
- It's great for feature selection as it can completely eliminate irrelevant features.
- Like Ridge, try different alpha values to optimize performance.

## **5. Model Evaluation Metrics**

##### **Calculating Mean Squared Error and R-squared**
[Code for calculating MSE and R-squared]

**Explanation**:
- MSE: Measures the average squared difference between predictions and actual values.
- R-squared: Represents the proportion of variance in the dependent variable that is predictable from the independent variable(s).

**Best Practices**:
- Use MSE to get an idea of the magnitude of your errors in the original unit of the target variable.
- R-squared is useful for comparing models, but be cautious of overfitting when R-squared is very close to 1.
- Always use a separate test set or cross-validation to get a true measure of model performance.




---


---

## **6. Using scikit-learn for Models with Three or More Features**

[Code for using scikit-learn models]

**Explanation**: Scikit-learn provides efficient, well-tested implementations of various machine learning algorithms.

**Best Practices**:
- Use scikit-learn for production code and when working with larger datasets.
- Take advantage of scikit-learn's consistent API across different models for easy experimentation.
- Use scikit-learn's built-in cross-validation and model selection tools for robust evaluation and tuning.

---

## **7. Summary and Insights**

This notebook demonstrates the fundamental concepts of linear regression, from basic implementation to more advanced techniques like regularization.

**Key Takeaways**:
1. Linear regression is a powerful tool for understanding relationships between variables.
2. Regularization techniques like Ridge and Lasso can help prevent overfitting and improve model generalization.
3. Visualizations are crucial for understanding data, model behavior, and results.
4. While implementing algorithms from scratch is educational, using established libraries like scikit-learn is recommended for real-world applications.

**Best Practices for Linear Regression**:
- Always start with exploratory data analysis to understand your data.
- Use simple models as a baseline before moving to more complex ones.
- Regularly check model assumptions (linearity, independence, homoscedasticity, normality of residuals).
- Be mindful of outliers and their impact on your model.
- Use cross-validation for more reliable performance estimates, especially with smaller datasets.
- Consider feature engineering to capture non-linear relationships if simple linear models aren't sufficient.


This revised notebook now includes explanations and best practices for each major concept, providing a more comprehensive understanding of linear regression and its applications. The added context will help learners not just implement the techniques, but also understand when and why to use them in real-world scenarios.